In [1]:
import gzip
import torch
import pandas as pd
from collections import defaultdict
import uuid
import numpy as np
from tqdm import tqdm
from torch.nn.functional import softmax, cross_entropy
import networkx as nx
import pickle
from torch import optim


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Matrix Bulid

In [3]:
ds_name = "Cadets" #Cadets, Theia
data_folder = f"dataset/{ds_name.lower()}/"
seq_len = 10 # set seq len


In [4]:
set_name = "train" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Cadets_train_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Cadets_train_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [5]:
set_name = "val" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Cadets_val_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Cadets_val_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [6]:
set_name = "test" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Cadets_test_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Cadets_test_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [7]:
ds_name = "Theia" #Cadets, Theia
data_folder = f"dataset/{ds_name.lower()}/"


In [8]:
set_name = "train" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Theia_train_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Theia_train_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [9]:
set_name = "val" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Theia_val_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Theia_val_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [10]:
set_name = "test" #train, val, test

fname_data_for_model = f"{ds_name}_{set_name}_data_for_model_one_hot_types.pkl"
fname_data_for_graph = f"{ds_name}_{set_name}_data_for_graph_one_hot_types.pkl"


Theia_test_data_for_model = pickle.load(open(data_folder+fname_data_for_model, "rb" ) )
Theia_test_data_for_graph = pickle.load(open(data_folder+fname_data_for_graph, "rb" ) )

In [11]:
# full_df_cadets = pd.read_csv("dataset\\alerted_events.csv")

# train_df_cadets = full_df_cadets.iloc[:6511, :]
# test_df_cadets = full_df_cadets.iloc[6511:, :]

# full_df_theia = pd.read_csv("dataset\\alerted_events_Theia.csv")
# train_df_theia = full_df_theia.iloc[:2900, :]
# test_df_theia = full_df_theia.iloc[2900:, :]

In [12]:
options = dict()
options['input_size'] =  46 #len(train_logs[0][0])
options['hidden_size'] = 16
options['num_layers'] = 2
options['num_classes'] = 6
options['batch_size'] = 64
options['p_dropout'] = 0.1
options['bidirectional'] = False
options['seq_len'] = 10

# Dataset

In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, Sampler
import numpy as np
from torch.utils.data import DataLoader

In [14]:
class log_dataset(Dataset):
    def __init__(self, data):
        self.data = data#['data_for_sequences']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        log = torch.tensor(self.data[idx]['logs_for_sequence'], dtype=torch.long)
        next_event_types = np.vstack(self.data[idx]['next_event_types']).astype(np.int)
        next_event_types = torch.tensor(next_event_types, dtype=torch.long)
        malicious_label = self.data[idx]['label']
        return log, next_event_types, malicious_label

In [15]:
both_train = Cadets_train_data_for_model['data_for_sequences'].copy()
both_train.extend(Theia_train_data_for_model['data_for_sequences'])
print('N of train sequences: {}'.format(len(both_train)))

N of train sequences: 9400


In [16]:
both_test = Cadets_test_data_for_model['data_for_sequences'].copy()
both_test.extend(Theia_test_data_for_model['data_for_sequences'])
print('N of test sequences: {}'.format(len(both_test)))

N of test sequences: 750


In [17]:
# # torch.from_numpy(Cadets_train_data_for_model['data_for_sequences'][0]['next_event_types'])
# import torch
# import numpy as np

# # Your test array without 'dtype=object'
# a = np.array([
#     np.array([0.5, 1.0, 2.0], dtype=np.float16),
#     np.array([4.0, 6.0, 8.0], dtype=np.float16),
# ])

# b = torch.from_numpy(a)

# print(a.dtype) # This should not be 'object'
# print(b)
a = Cadets_train_data_for_model['data_for_sequences'][0]['next_event_types']

In [18]:
train_set = log_dataset(both_train)
train_loader = DataLoader(train_set,
                               batch_size=1,#options['batch_size'],
                               shuffle=True,
                               pin_memory=True)


test_set = log_dataset(both_test)
test_loader = DataLoader(test_set,
                               batch_size=1,#options['batch_size'],
                               shuffle=True,
                               pin_memory=True)

# from sklearn.utils.class_weight import compute_class_weight

# weights = compute_class_weight(class_weight='balanced',classes=np.unique(train_df['type']),y=train_df['type'])
# weights = torch.Tensor(weights)
# weights.to(device)

In [19]:
# input_example = next(iter(train_loader))[0]
# input_example.shape

# Model

In [20]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class EncoderLSTM(nn.Module):
    def __init__(self, input_size=options['input_size'], hidden_size=options['hidden_size']):#p_dropout, bidirectional, num_layers, num_classes
        super(EncoderLSTM, self).__init__()
#         self.bidirectional = bidirectional
        self.hidden_size = input_size#hidden_size
#         self.num_layers = num_layers
#         self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.GRU(input_size=input_size,
                            hidden_size=self.hidden_size,)
#                             num_layers=num_layers,
#                             batch_first=True,
#                            dropout=p_dropout,
#                            bidirectional=bidirectional)
#         self.fc = nn.Linear(num_layers*2*hidden_size, num_classes)

    def forward(self, one_log_event, hidden):
#         embedded = self.embedding(one_log_event).view(1, 1, -1)
        output = one_log_event.view(1, 1, -1).float()#embedded #
#         print(one_log_event.shape, output.shape, hidden.shape)
        output, hidden = self.lstm(output, hidden)
#         for log in logs:
#             _, hid = self.lstm(log.float(), hid)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device) #1, num_layers, self.hidden_size
    
class AttnDecoderLSTM(nn.Module):
    def __init__(self, hidden_size=options['input_size'], output_size=options['num_classes'], 
                 dropout_p=options['p_dropout'], seg_length=options['seq_len']): #hidden_size=options['hidden_size']
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p

#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.encoder_out_to_output_size = nn.Linear(self.hidden_size, output_size)
        self.attn = nn.Linear(hidden_size + output_size, seg_length)
        self.attn_combine = nn.Linear(hidden_size + output_size, hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.GRU(hidden_size, hidden_size)
        self.out1 = nn.Linear(hidden_size, hidden_size//2)

        self.out2 = nn.Linear(hidden_size//2, output_size)

    def forward(self, input, hidden, encoder_outputs):
        hidden = hidden.view(1, 1, -1)
        input_embedded = input.view(1, 1, -1)#
        attn_weights = F.softmax(
            self.attn(torch.cat((input_embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((input_embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out1(output[0])
        output = F.log_softmax(self.out2(output), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [21]:
encoder = EncoderLSTM().to(device)
decoder = AttnDecoderLSTM().to(device)

In [22]:
train_set[0]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11544\2809914774.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  next_event_types = np.vstack(self.data[idx]['next_event_types']).astype(np.int)


(tensor([[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
          0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
          0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
          0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
          0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
          0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [23]:
# input_example = next(iter(train_loader))[0]
# input_example.squeeze().shape  #squeeze() because we have first dimension who is redundent

In [24]:
# #labels:
# example_labels = next(iter(train_loader))[1]
# example_labels.shape, example_labels

In [25]:
"""
This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.
"""
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [49]:
EPOCHS = 50 #300 #
# optimizer = torch.optim.Adam(model.parameters(),
#                             lr=0.0001,
#                             betas=(0.9, 0.999))
lr=0.0001
criterion = nn.NLLLoss()#weight = torch.Tensor(weights).to(device))
# val_criterion = torch.nn.CrossEntropyLoss()
num_batch = len(train_loader)

history = {'acc': [], 'epoch': [], 'loss': []}
history_test = {'acc': [], 'epoch': [], 'loss': []}

encoder_optimizer = optim.SGD(encoder.parameters(), lr=lr)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=lr)

# train

In [50]:
SOS_tok = torch.zeros(6, dtype=torch.long, device=device)

In [51]:
a5 = torch.zeros(6, dtype=torch.long, device=device)
a5[[3]] = 1
a5

tensor([0, 0, 0, 1, 0, 0])

In [52]:
SOS_tok = torch.zeros(6, dtype=torch.long, device=device)
from tqdm import tqdm


import torch.nn.functional as F

start = time.time()

        
for i in range(EPOCHS):
    encoder.train()
    decoder.train()
    epoch_loss = 0
    epoch_acc = 0
    print(f'epoch {i}/{EPOCHS}:')
    print_loss_total = 0 
    # pbar = tqdm(enumerate(train_loader))
    for idx, (train_logs, train_labels, malicious_label) in enumerate(train_loader):   

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(options['seq_len'], encoder.hidden_size, device=device)
        loss = 0
        
        for ei in train_logs.squeeze():
            encoder_output, encoder_hidden = encoder(ei, encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
        decoder_input = SOS_tok
        decoder_hidden = encoder_hidden
        use_teacher_forcing = True

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for ind, di in enumerate(train_labels.squeeze()):
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                loss += criterion(decoder_output.squeeze(), di)
                decoder_input = torch.zeros(6, dtype=torch.long, device=device)  
                decoder_input[di] = 1  # Teacher forcing
        
        else:
            # Without teacher forcing: use its own predictions as the next input
            for ind, di in enumerate(train_labels.squeeze()):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                loss += criterion(decoder_output.squeeze(), di)
                
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        print_loss_total += loss.item() / options['seq_len']
    
    print(f"{timeSince(start, 1)} loss: {print_loss_total/len(train_loader)}")
    
    #validation:
    encoder.eval()
    decoder.eval()
    epoch_loss = 0
    epoch_acc = 0 
    with torch.no_grad():
        for idx, (test_logs, test_labels, malicious_label) in enumerate(test_loader):
#             output = model(test_logs).to(device)
            encoder_outputs = torch.zeros(options['seq_len'], encoder.hidden_size, device=device)
            loss = 0

            for ei in test_logs.squeeze():
                encoder_output, encoder_hidden = encoder(ei, encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = SOS_tok
            decoder_hidden = encoder_hidden
            use_teacher_forcing = True

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for ind, di in enumerate(test_labels.squeeze()):
                    decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                    loss += criterion(decoder_output.squeeze(), di)
                    decoder_input = torch.zeros(6, dtype=torch.long, device=device)  
                    decoder_input[di] = 1  # Teacher forcing

            else:
                # Without teacher forcing: use its own predictions as the next input
                for ind, di in enumerate(test_labels.squeeze()):
                    decoder_output, decoder_hidden, decoder_attention = decoder(
                        decoder_input, decoder_hidden, encoder_outputs)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output.squeeze(), di)

            
#             acc = (softmax(output, -1).argmax(-1) == label).float().sum()#categorical_accuracy(output, label)

            epoch_loss += loss.item()
#             epoch_acc += acc.item()/options['seq_len']
                # pbar.set_description(f"{len(train_loader)}/{idx}")
    history_test['epoch'].append(i+1)
    history_test['loss'].append(epoch_loss/(idx+1))
#     history_test['acc'].append(epoch_acc/(idx+1))
    print(f'Mode:Test | Epoch {i+0:03}: | Loss: {epoch_loss/len(test_loader):.5f}')# | Acc: {epoch_acc/(idx+1):.3f}%')



epoch 0/50:


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11544\2809914774.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  next_event_types = np.vstack(self.data[idx]['next_event_types']).astype(np.int)


5m 15s (- 0m 0s) loss: 0.09636356657208585
Mode:Test | Epoch 000: | Loss: 16.82724
epoch 1/50:
10m 38s (- 0m 0s) loss: 0.09351284534895207
Mode:Test | Epoch 001: | Loss: 15.74095
epoch 2/50:
15m 39s (- 0m 0s) loss: 0.09079783341674597
Mode:Test | Epoch 002: | Loss: 14.71578
epoch 3/50:
18m 52s (- 0m 0s) loss: 0.08860196740711695
Mode:Test | Epoch 003: | Loss: 13.91779
epoch 4/50:
22m 20s (- 0m 0s) loss: 0.08634088917804408
Mode:Test | Epoch 004: | Loss: 12.98874
epoch 5/50:
25m 44s (- 0m 0s) loss: 0.08384029366430179
Mode:Test | Epoch 005: | Loss: 12.79223
epoch 6/50:
29m 9s (- 0m 0s) loss: 0.08240062626629176
Mode:Test | Epoch 006: | Loss: 12.04663
epoch 7/50:
32m 33s (- 0m 0s) loss: 0.08007081454693388
Mode:Test | Epoch 007: | Loss: 11.65128
epoch 8/50:
36m 2s (- 0m 0s) loss: 0.07837147076762034
Mode:Test | Epoch 008: | Loss: 11.26966
epoch 9/50:
39m 29s (- 0m 0s) loss: 0.07689446571026448
Mode:Test | Epoch 009: | Loss: 11.42342
epoch 10/50:
42m 55s (- 0m 0s) loss: 0.0755911111689958

In [54]:
# import datetime
# time_of_saving = str(datetime.datetime.now())[:-13]
# print(time_of_saving)
# torch.save(encoder.state_dict(), f"{time_of_saving} - encoder_model cadets and theia seq2seq2.pth")
# torch.save(decoder.state_dict(), f"{time_of_saving} - decoder_model cadets and theia seq2seq2.pth")

2023-07-18 10


In [48]:
# torch.cat((decoder_output.view(1,-1), decoder_output.view(1,-1)))
torch.tensor([1,4,5,6])

tensor([1, 4, 5, 6])

In [55]:
# from tqdm import tqdm

# for i in range(EPOCHS):
#     model.train()
#     sum_loss = 0.0
#     total = 0
#     epoch_loss = 0
#     epoch_acc = 0
#     hid = None
#     # pbar = tqdm(enumerate(train_loader))
#     for idx, (train_logs, label, malicious_label) in enumerate(train_loader):      
#         output = model(train_logs).to(device)
#         # we transpose the predictions since crossentropy loss
#         # expects to receive for the prediction a tensor shape (Batch, Classes, d1, d2, ....)
#         # the output of the LSTM is shape (Batch, timesteps, classes)
#         # so in order to have the correct format we need to transpose it to shape (Batch, classes, timesteps)
#         # y is derived from batch which is a tensor of one-hot
#         # crossentropy expects a categorical representation and not one-hot
#         # so to extract the classes themselves we use argmax.
#         print(output, label)
#         loss = criterion(output.transpose(-1, -2), label)

#         optimizer.zero_grad()
#         acc = (softmax(output, -1).argmax(-1) == label).float().sum()#categorical_accuracy(output, label)

#         epoch_loss += loss.item()
#         epoch_acc += acc.item()/options['seq_len']
#         loss.backward()

#         optimizer.step()
#         # pbar.set_description(f"{len(train_loader)}/{idx}")
#     history['epoch'].append(i+1)
#     history['loss'].append(epoch_loss/len(train_loader))
#     history['acc'].append(epoch_acc/len(train_loader))
#     print(f'Mode:Train | Epoch {i+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}%')            
                   
#     hid = None
#     model.eval()
#     sum_loss = 0.0
#     total = 0
#     epoch_loss = 0
#     epoch_acc = 0 
#     with torch.no_grad():
#         for idx, (test_logs, label, malicious_label) in enumerate(test_loader):
#             output = model(test_logs).to(device)

#             loss = criterion(output.transpose(-1, -2), label)#, weight=weights
#             acc = (softmax(output, -1).argmax(-1) == label).float().sum()#categorical_accuracy(output, label)

#             epoch_loss += loss.item()
#             epoch_acc += acc.item()/options['seq_len']
#                 # pbar.set_description(f"{len(train_loader)}/{idx}")
#     history_test['epoch'].append(i+1)
#     history_test['loss'].append(epoch_loss/(idx+1))
#     history_test['acc'].append(epoch_acc/(idx+1))
#     print(f'Mode:Test | Epoch {i+0:03}: | Loss: {epoch_loss/len(test_loader):.5f} | Acc: {epoch_acc/(idx+1):.3f}%')

#     # if i > 0 and history_test['loss'][i-1] < history_test['loss'][i]:
#     #     break


In [56]:
# import matplotlib.pyplot as plt

# plt.plot(history['loss'], label='train loss')
# plt.plot(history_test['loss'], label='validation loss')

# plt.legend()

# plt.show()


In [57]:
# import matplotlib.pyplot as plt

# plt.plot(history['acc'], label='train acc')
# plt.plot(history_test['acc'], label='validation acc"')

# plt.legend()

# plt.show()


In [58]:
# import matplotlib.pyplot as plt

# plt.plot(history['epoch'], history['acc'], label='accuracy')
# plt.plot(history['epoch'], history['loss'], label='loss')
# #plt.plot(validationhistory)
# plt.title('Training')
# plt.legend()
# plt.show()

# Evaluations

In [63]:
both_val = Cadets_val_data_for_model['data_for_sequences'].copy()
both_val.extend(Theia_val_data_for_model['data_for_sequences'])
print('N of val sequences: {}'.format(len(both_val)))

N of val sequences: 210


## Find anomaly threshold from validation set

In [64]:
SOS_tok

tensor([0, 0, 0, 0, 0, 0])

In [65]:
# anomaly_score_ls = []
mistakes_ls = []
candidates = 1


with torch.no_grad():

    for seq in tqdm(both_val):
        ind = seq['seq_ind']
        logs_input = torch.tensor([seq['logs_for_sequence']], dtype=torch.long).to(device)
        next_event_types = torch.tensor(seq['next_event_types'], dtype=torch.long).long().to(device)
        
        
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(options['seq_len'], encoder.hidden_size, device=device)
        
        for ei in logs_input.squeeze():
            encoder_output, encoder_hidden = encoder(ei, encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
        decoder_input = SOS_tok
        decoder_hidden = encoder_hidden
        use_teacher_forcing = False
        
        mistakes = options['seq_len']
#         print(labels.squeeze(), desired_i)

        for ind, di in enumerate(next_event_types.squeeze()):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

            topv, topi = decoder_output.topk(1)
#             print(decoder_output.topk(5)[0])
            decoder_output = topi.squeeze().detach()  # detach from history as input
            desired_i = di
#             print("di", di, "desired_i", desired_i)
            if desired_i == decoder_output:
                mistakes-=1
            
            decoder_input = torch.zeros(6, dtype=torch.long, device=device)  
            if use_teacher_forcing: decoder_input[di] = 1  # Teacher forcing
            else: decoder_input[decoder_output] = 1
            
        mistakes_ls.append(mistakes)
        decoded_words.append(decoder_output)
        
        
#         logs_input = torch.tensor([logs_input], dtype=torch.float).to(device)
#         next_event_types = torch.tensor(next_event_types, dtype=torch.float).long().to(device)
#         output = model(logs_input)
        
#         soft_out = softmax(output, dim=-1).squeeze()
#         sorted_preds = torch.argsort(output.squeeze(), dim=-1 ,descending=True)
#         anomaly_score = 0
#         for ind_event, pred, correct_event_type in zip(range(seq_len), sorted_preds, next_event_types):
#             top_k_preds = pred[:candidates] 
#             if correct_event_type not in top_k_preds: 
#                 anomaly_score += 0.2 * (1-soft_out[ind_event, correct_event_type])
#         anomaly_score_ls.append(anomaly_score)


100%|███████████████████████████████████████████████████████████████████████████████| 210/210 [00:00<00:00, 418.26it/s]


In [66]:
np.average(mistakes_ls), np.max(mistakes_ls)

(0.2857142857142857, 7)

## Evaluate test set

In [71]:
encoder.eval()
decoder.eval()

FP = 0
FN = 0
TP = 0
TN = 0

FP_ls = []
TP_ls = []


# anomaly_score_ls = []
threshold_mistakes = 1
mistakes_ls = []
# candidates = 1


with torch.no_grad():

    for seq in tqdm(both_test):
        ind = seq['seq_ind']
        logs_input = torch.tensor([seq['logs_for_sequence']], dtype=torch.long).to(device)
        next_event_types = torch.tensor(seq['next_event_types'], dtype=torch.long).long().to(device)
        malicious_label = seq['label']
        
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(options['seq_len'], encoder.hidden_size, device=device)
        
        for ei in logs_input.squeeze():
            encoder_output, encoder_hidden = encoder(ei, encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
        decoder_input = SOS_tok
        decoder_hidden = encoder_hidden
        use_teacher_forcing = False
        
        mistakes = options['seq_len']
#         print(labels.squeeze(), desired_i)

        for ind, di in enumerate(next_event_types.squeeze()):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

            topv, topi = decoder_output.topk(1)
#             print(decoder_output.topk(5)[0])
            decoder_output = topi.squeeze().detach()  # detach from history as input
            desired_i = di
#             print("di", di, "desired_i", desired_i)
            if desired_i == decoder_output:
                mistakes-=1
            
            decoder_input = torch.zeros(6, dtype=torch.long, device=device)  
            if use_teacher_forcing: decoder_input[di] = 1  # Teacher forcing
            else: decoder_input[decoder_output] = 1
                
        mistakes_ls.append(mistakes)
        decoded_words.append(decoder_output)
        

        
        if malicious_label == 1: # malicious
            if mistakes<threshold_mistakes:
                FN += 1
#                 print(f"FN: seq ind-{ind} mistakes:{mistakes}")

            else:
#                 print(f"TP: seq ind-{ind} mistakes:{mistakes}")
                TP_ls.append(ind)
                TP += 1
        elif malicious_label == 0: # benign
            if mistakes<threshold_mistakes:
                TN += 1
#                 print(f"TN: seq ind-{ind} mistakes:{mistakes}")
            else:
#                 print(f"FP: seq ind-{ind} mistakes:{mistakes}")
                FP_ls.append(ind)
                FP += 1


100%|███████████████████████████████████████████████████████████████████████████████| 750/750 [00:01<00:00, 403.28it/s]


In [72]:
FPR = FP/(FP+TN)
P = TP / (TP + FP) if TP + FP>0 else 0
R = TP / (TP + FN) if TP + FN>0 else 0
print('true positive: {}, \nfalse positive (FP): {}, \ntrue negative: {}, \nfalse negative (FN): {}, \nPrecision: {:.3f}, \nRecall: {:.3f}, \nFalse positive rate: {:.3f}'
    .format(TP, FP, TN, FN, P, R, FPR))

true positive: 43, 
false positive (FP): 214, 
true negative: 493, 
false negative (FN): 0, 
Precision: 0.167, 
Recall: 1.000, 
False positive rate: 0.303


# Check metrics:

# second approach to anomaly - 0.2*softmax of truth event

In [36]:
FP = 0
FN = 0
TP = 0
TN = 0
# anomaly_labels = test_df['anomaly_label'][10:]
hid = None

threshold_anomaly_score = 0.76 #how many mistakes allowd in preds for a sequence to count as a benign 
seq_len = options['seq_len']
candidates = 1

pbar = tqdm(test_set, total=len(test_set), position=0, leave=True)

with torch.no_grad():
    for idx, (test_logs, label, malicious_label) in enumerate(test_loader):   
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(options['seq_len'], encoder.hidden_size, device=device)
        
        for ei in test_logs.squeeze():
            encoder_output, encoder_hidden = encoder(ei, encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
        SOS_tok = torch.zeros(1, options['input_size'], device=device)
        decoder_input = SOS_tok
        decoder_hidden = encoder_hidden

        
#     for seq_ind, sequence in enumerate(pbar):
#         logs = sequence[0].unsqueeze(0) #add dimension to make it batch of one
#         seq = torch.tensor(logs, dtype=torch.float).to(device)
#         label = torch.tensor(sequence[1], dtype=torch.float).long().to(device)
#         output = model(seq)
# #         predicted = torch.argsort(output,1)[0][-candidates:]
#         soft_out = softmax(output, dim=-1).squeeze()
#         sorted_preds = torch.argsort(output.squeeze(), dim=-1 ,descending=True)#[:,:3] 
#         anomaly_score = 0
#         for ind_event, pred, correct_event_type in zip(range(seq_len), sorted_preds, label):
#             top_k_preds = pred[:candidates] 
#             if correct_event_type not in top_k_preds: 
#                 anomaly_score += 0.2 * (1-soft_out[ind_event, correct_event_type])
# #         event_id_for_seq = event_ids_test[seq_ind]
#         malicious_label = sequence[2]
#         if malicious_label == 1: # anomaly
#             #print("anomaly")
#             if anomaly_score<threshold_anomaly_score:
#                 print(f"seq ind - {seq_ind}: anomaly_score - {anomaly_score} -------FN-------- ")#event_ids - {event_id_for_seq}")
#                 FN += 1
#             else:
#                 print(f"seq ind - {seq_ind}: anomaly_score - {anomaly_score} -------TP ")#event_ids - {event_id_for_seq}")
#                 TP += 1
#         elif malicious_label == 0: # benign
#             if anomaly_score<threshold_anomaly_score:
#                 TN += 1
#             else:
#                 print(f"seq ind - {seq_ind}: anomaly_score - {anomaly_score} FP------- ")#event_ids - {event_id_for_seq}")
#                 FP += 1


  0%|                                                                                          | 0/750 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_11544\2809914774.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  next_event_types = np.vstack(self.data[idx]['next_event_types']).astype(np.int)


In [37]:
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
FPR = FP/(FP+TP)
# F1 = 2 * P * R / (P + R)
print('true positive: {}, \nfalse positive (FP): {}, \ntrue negative: {}, \nfalse negative (FN): {}, \nPrecision: {:.3f}%, \nRecall: {:.3f}%, , \nFalse positive rate: {:.3f}'
    .format(TP, FP, TN, FN, P, R, FPR))

ZeroDivisionError: division by zero

In [ ]:
# fp_closed_to_alerts = 22
# new_FP = FP-fp_closed_to_alerts
# FPR = new_FP/(new_FP+TP)
# P = 100 * TP / (TP + new_FP) if TP + new_FP>0 else 0
# R = 100 * TP / (TP + FN) if TP + FN>0 else 0
# print('true positive: {}, \nfalse positive (FP): {}, \ntrue negative: {}, \nfalse negative (FN): {}, \nPrecision: {:.3f}%, \nRecall: {:.3f}%, , \nFalse positive rate: {:.3f}'
#     .format(TP, new_FP, TN, FN, P, R, FPR))

In [ ]:
# import datetime
# time_of_saving = str(datetime.datetime.now())[:-13]
# print(time_of_saving)
# torch.save(model.state_dict(), f"{time_of_saving} - agregator_model cadets and theia 0.3 dropout.pth")